Sets:

Let $ V = \{1, 2, \dots, n\} $ be the set of vehicles (taxis), each with capacity $ C_v $ for $ v \in V $.
Let $ R = \{1, 2, \dots, m\} $ be the set of passenger requests.
For each request $ r \in R $, define:

Pickup node $ p_r $ and drop-off node $ d_r $.
Time window for pickup: $ [e_r, l_r] $ (earliest/latest time).
Direct distance/time: $ dist(p_r, d_r) $, $ time(p_r, d_r) $.


Let $ N = \{o_v, f_v \mid v \in V\} \cup \{p_r, d_r \mid r \in R\} $ be all nodes (origins/depots $ o_v $, final depots $ f_v $, pickups, drop-offs).
Let $ A $ be the set of arcs (i,j) with travel time $ t_{ij} $ and distance $ d_{ij} $ (from graph G).

Variables:

$ x_{ijk} \in \{0,1\} $: 1 if vehicle $ k $ traverses arc (i,j), for $ i,j \in N $, $ k \in V $.
$ T_{ik} \geq 0 $: Arrival time of vehicle $ k $ at node $ i $.
$ Q_{ik} \geq 0 $: Load (number of passengers) in vehicle $ k $ after visiting node $ i $.
$ y_{rk} \in \{0,1\} $: 1 if request $ r $ is assigned to vehicle $ k $ (optional for assignment).

Objective:
Minimize total cost (e.g., distance + penalties for wait/detour):
$$\min \sum_{k \in V} \sum_{(i,j) \in A} d_{ij} x_{ijk} + \alpha \sum_{r \in R} \max(0, T_{d_r k} - T_{p_r k} - time(p_r, d_r)) + \beta \sum_{r \in R} \max(0, T_{p_r k} - e_r)$$
(where $ \alpha, \beta $ are weights for detour/wait penalties).
Constraints:

Flow conservation (each node visited exactly once per assigned vehicle):

$$\sum_{j \in N} x_{jik} = \sum_{j \in N} x_{ijk} \quad \forall i \in N \setminus \{o_k, f_k\}, k \in V$$

Vehicle starts/ends at depot:

$$\sum_{j \in N} x_{o_k j k} = 1, \quad \sum_{i \in N} x_{i f_k k} = 1 \quad \forall k \in V$$

Assignment: Each request assigned to exactly one vehicle:

$$\sum_{k \in V} y_{rk} = 1 \quad \forall r \in R$$

Link assignment to flow (pickup/drop-off served by same vehicle):

$$\sum_{j \in N} x_{p_r j k} = y_{rk}, \quad \sum_{j \in N} x_{d_r j k} = y_{rk} \quad \forall r \in R, k \in V$$

Precedence (pickup before drop-off):

$$T_{d_r k} \geq T_{p_r k} + t_{p_r d_r} + M (1 - y_{rk}) \quad \forall r \in R, k \in V$$
(where M is large constant).
6. Time propagation:
$$T_{jk} \geq T_{ik} + t_{ij} - M (1 - x_{ijk}) \quad \forall i,j \in N, k \in V$$

Time windows:

$$e_r \leq T_{p_r k} \leq l_r \quad \forall r \in R, k \in V \ (if assigned)$$
(with big-M for non-assigned).
8. Capacity:
$$Q_{jk} = Q_{ik} + s_i - M (1 - x_{ijk}) \quad \forall i,j \in N, k \in V$$
(where $ s_i = 1 $ for pickup, -1 for drop-off, 0 else; $ 0 \leq Q_{ik} \leq C_v $).
9. Detour limit (e.g., ride time ≤ γ * direct time):
$$T_{d_r k} - T_{p_r k} \leq \gamma \cdot time(p_r, d_r) + M (1 - y_{rk}) \quad \forall r \in R, k \in V$$
This is a standard MILP formulation for dynamic DARP; solve with PuLP/Gurobi. For real-time, use anytime solving or heuristics.

In [ ]:
import osmnx as ox
import networkx as nx
import random
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Config (de votre code)
PLACE = "Bordj El Bahri, Algeria"
NUM_DRIVERS = 5
CAPACITY = 3
MAX_WAIT = 300  # s
DETOUR_MAX = 1.2
NUM_PASSENGERS = 5  # Exemple batch

# Charger le graphe (comme dans votre code)
G = ox.graph_from_place(PLACE, network_type="drive")
G = G.to_undirected()
nodes = list(G.nodes)

def compute_time_matrix(G, locations):
    """Calcule la matrice de temps (s) entre locations (liste de nœuds)."""
    n = len(locations)
    time_matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            try:
                path = nx.shortest_path(G, locations[i], locations[j], weight='length')
                time_matrix[i][j] = route_time(G, path)  # Utilisez votre fonction route_time
            except nx.NetworkXNoPath:
                time_matrix[i][j] = float('inf')
    return time_matrix

def create_data_model(G, drivers, passengers):
    """Prépare les données pour OR-Tools."""
    data = {}
    
    # Locations: depot (dummy, utilisez position actuelle des taxis), pickups, dropoffs
    # Pour dynamique: depot = positions actuelles des taxis (multi-depots)
    depots = [d.node for d in drivers]  # Positions actuelles des taxis comme starts
    pickups = [p.origin for p in passengers if p.status == "waiting"]
    dropoffs = [p.dest for p in passengers if p.status == "waiting"]
    
    # Tous les nœuds uniques (indexés)
    all_locations = list(set(depots + pickups + dropoffs))
    node_to_idx = {node: idx for idx, node in enumerate(all_locations)}
    
    data['time_matrix'] = compute_time_matrix(G, all_locations)  # Matrice temps (s)
    data['pickups_deliveries'] = [[node_to_idx[p.origin], node_to_idx[p.dest]] for p in passengers if p.status == "waiting"]
    data['num_vehicles'] = len(drivers)
    data['starts'] = [node_to_idx[d.node] for d in drivers]  # Multi-starts (positions actuelles)
    data['ends'] = [node_to_idx[random.choice(nodes)] for _ in drivers]  # Ends aléatoires ou idle
    data['vehicle_capacities'] = [CAPACITY for _ in drivers]
    data['demands'] = [0] * len(all_locations)  # Demandes: +1 pickup, -1 dropoff
    for pickup, delivery in data['pickups_deliveries']:
        data['demands'][pickup] = 1
        data['demands'][delivery] = -1
    
    # Fenêtres temporelles (pour MAX_WAIT)
    current_time = 0  # Remplacez par CURRENT_TIME
    data['time_windows'] = [(0, float('inf'))] * len(all_locations)
    for i, (pickup, _) in enumerate(data['pickups_deliveries']):
        data['time_windows'][pickup] = (current_time, current_time + MAX_WAIT)  # Fenêtre pour pickup
    
    return data, node_to_idx

def print_solution(data, manager, routing, solution, drivers, passengers, node_to_idx):
    """Affiche et met à jour les routes des drivers."""
    idx_to_node = {v: k for k, v in node_to_idx.items()}
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan = []
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            plan.append(idx_to_node[node])
            index = solution.Value(routing.NextVar(index))
        plan.append(idx_to_node[manager.IndexToNode(index)])  # End
        
        # Mettre à jour le driver
        drv = drivers[vehicle_id]
        drv.route = plan
        # Mettre à jour types/passengers via matching avec pickups/deliveries
        # (Ajoutez logique pour identifier pickups/dropoffs)
        
        route_time = solution.ObjectiveValue()  # Ou calculez via route_time(G, plan)
        total_time += route_time
        print(f"Route for {drv.id}: {plan} (time: {route_time}s)")
    
    print(f"Total time: {total_time}s")

def solve_with_ortools(G, drivers, passengers, current_time):
    data, node_to_idx = create_data_model(G, drivers, passengers)
    
    manager = pywrapcp.RoutingIndexManager(
        len(data['time_matrix']), data['num_vehicles'], data['starts'], data['ends']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # Callback temps
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['time_matrix'][from_node][to_node])
    
    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Dimension Temps (pour fenêtres et détours)
    routing.AddDimension(
        transit_callback_index,
        int(MAX_WAIT),  # Slack max (attente)
        3600,  # Max temps par véhicule (1h)
        False,  # Ne pas fixer cumul à 0
        'Time'
    )
    time_dimension = routing.GetDimensionOrDie('Time')
    for i, tw in enumerate(data['time_windows']):
        if tw[0] != 0 or tw[1] != float('inf'):
            index = manager.NodeToIndex(i)
            time_dimension.CumulVar(index).SetRange(int(tw[0]), int(tw[1]))
    
    # Dimension Capacité
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # No slack
        data['vehicle_capacities'],
        True,  # Start à 0
        'Capacity'
    )
    
    # Pickups et Deliveries
    for pickup, delivery in data['pickups_deliveries']:
        routing.AddPickupAndDelivery(pickup, delivery)
        routing.solver().Add(routing.VehicleVar(pickup) == routing.VehicleVar(delivery))
        routing.solver().Add(time_dimension.CumulVar(pickup) <= time_dimension.CumulVar(delivery))
    
    # Heuristique et Recherche
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(10)  # Limite temps
    
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(data, manager, routing, solution, drivers, passengers, node_to_idx)
    else:
        print("Pas de solution trouvée.")

# Exemple d'appel (intégrez dans update())
# solve_with_ortools(G, drivers, passengers, CURRENT_TIME)